In [76]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [77]:
# Import tensorflow dependencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [40]:
# # n=int(input("enter no of entries:"))

# # for i in range(n):
# #     name=input("ENTER THE NAME OF THE PERSON:")
# #     POS_PATH = os. path.join('dataset',name, 'positive')
# #     NEG_PATH = os. path.join('dataset',name, 'negative')
# #     ANC_PATH = os. path.join('dataset',name, 'anchor')

# #     # making those folders
# #     os.makedirs(NEG_PATH)
# #     os.makedirs(POS_PATH)
# #     os.makedirs(ANC_PATH)
# #     #os.makedirs(RS_PATH)



#     name = str(input("Enter Name :: ")+'_')
#     cap = cv2.VideoCapture(0)
#     j = 0
#     k = 0
#     while cap.isOpened():
#         ret,fram = cap.read()
#         fram = fram[235:235+250,515:250+515]
#         cv2.imshow('Image Collection',fram)
#         if cv2.waitKey(1) & 0XFF == ord('a'):
#             print(f"Selfie saved ")

#             # Create the unique file path
#             i = str(j)
#             name_anc = name + i
#             imgname = os.path.join (ANC_PATH, '{}.jpg'.format(name_anc))
#             j = j + 1
#             # Write out anchor image
#             cv2.imwrite(imgname, fram)
        
#         elif cv2.waitKey(1) & 0XFF == ord('p'):
#             # Create the unique file path
#             # k = 0
#             p = str(k)
#             name_pos = name + p
#             imgname = os. path.join (POS_PATH, '{}.jpg'.format(name_pos))
#             k = k + 1
#             # Write out Positive image
#             cv2.imwrite(imgname, fram)
                
#         elif cv2.waitKey(1) & 0XFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows() 
    

# for directory in os.listdir('lfw'):
#     for file in os.listdir(os.path.join('lfw', directory)):
#         EX_PATH = os.path.join('lfw', directory, file)
#         NEW_PATH = os.path.join(NEG_PATH, file)
#         os.replace(EX_PATH, NEW_PATH)
    
# print("All entries processed.")

   



In [78]:
# name = str(input("Enter Name :: ")+'_')
# name  = capitalize(name)
# # ANC_PATH = "/Users/DEVDESAI1/Desktop/Projects/Face_Recognision/Training/" + name + "/anchor"
# # POS_PATH = "/Users/DEVDESAI1/Desktop/Projects/Face_Recognision/Training/" + name + "/positive"
count = 0
names=['amisha','atish','ayush','balaji','manish','raja','vatsal','misbah']
for name in names:
    ANC_PATH = "dataset/"+ name +"/anchor"
    POS_PATH = "dataset/" + name + "/positive" 
    NEG_PATH = "dataset/" + name + "/negative"
    anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(200)
    positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(200)
    negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(200)
    positives = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
    negatives = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
    if count == 1:
        data = data.concatenate(negatives)
        
        data = data.concatenate(positives)
        # data = data.shuffle(
    elif count == 0:
        data = positives.concatenate(negatives)
        count=1

In [79]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (100,100))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

In [80]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

In [81]:
# Build dataloader pipeline
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=10000)


In [82]:
# Training partition
train_data = data.take(round(len(data)*.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [83]:
# Testing partition
test_data = data.skip(round(len(data)*.7))
test_data = test_data.take(round(len(data)*.3))
test_data = test_data.batch(16)
test_data = test_data.prefetch(8)

In [84]:
def make_embedding():
    inp=Input(shape=(100,100,3), name='input_image')
    
    #first block
    c1=Conv2D(64,(10,10),activation='relu')(inp)
    m1=MaxPooling2D(64,(2,2),padding='same')(c1)
    
    #second block
    c2=Conv2D(128,(7,7),activation='relu')(m1)
    m2=MaxPooling2D(64,(2,2),padding='same')(c2)
    
    #third block
    c3=Conv2D(128,(4,4),activation='relu')(m2)
    m3=MaxPooling2D(64,(2,2),padding='same')(c3)
    
    #final embedding block
    c4=Conv2D(256,(4,4), activation='relu')(m3)
    f1=Flatten()(c4)
    d1=Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [85]:
embedding=make_embedding()

In [86]:
#siamese L1 Distance class 
class L1Dist(Layer):
    
    #init method - inheritance
    def __init__(self,**kwargs):
        super(). __init__()
        
    #similarity calculation
    def call(self,input_embedding,validation_embedding):
        return tf.math.abs(input_embedding-validation_embedding)
    

In [3]:
l1=L1Dist()


NameError: name 'L1Dist' is not defined

In [4]:
def make_siamese_model():
    
    #anchor image input in the network
    input_image=Input(name='input_img', shape=(100,100,3))
    
    #validation image in the network
    validation_image=Input(name='validation_img',shape=(100,100,3))
    
    #combine siamese distance components
    siamese_layer=L1Dist()
    siamese_layer._name='distance' # reason for dot .
    distances =siamese_layer(embedding(input_image),embedding(validation_image))
    
    #classification layer
    classifier=Dense(1,activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier ,name='SiameseNetwork')

In [89]:
siamese_model = make_siamese_model()

In [90]:
binary_cross_loss = tf.losses.BinaryCrossentropy()

In [91]:
opt = tf.keras.optimizers.Adam(1e-4) # 0.0001

In [92]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

In [93]:
@tf.function
def train_step(batch):
    
    # Record all of our operations 
    with tf.GradientTape() as tape:     
        # Get anchor and positive/negative image
        X = batch[:2]
        # Get label
        y = batch[2]
        
        # Forward pass
        yhat = siamese_model(X, training=True)
        # Calculate loss
        loss = binary_cross_loss(y, yhat)
    print(loss)
        
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
        
    # Return loss
    return loss

In [94]:
# Import metric calculations
from tensorflow.keras.metrics import Precision, Recall

In [96]:
def train(data, EPOCHS):
    # Loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch, EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        # Creating a metric object 
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            # Run train step here
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2])
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat) 
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
        
        # Save checkpoints
        if epoch % 10 == 0: 
            checkpoint.save(file_prefix=checkpoint_prefix)

In [22]:
#EPOCHS = 50

In [23]:
#train(train_data, EPOCHS)


 Epoch 1/50
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
Tensor("binary_crossentropy/weighted_loss/value:0", shape=(), dtype=float32)
140/140 [==============================] - 1204s 9s/step
0.41745448 0.41864717 0.83272725

 Epoch 2/50
140/140 [==============================] - 1201s 9s/step
0.51416665 0.7333916 0.8364905

 Epoch 3/50
140/140 [==============================] - 2286s 16s/step
0.33981735 0.7477477 0.8434959

 Epoch 4/50
140/140 [==============================] - 1215s 9s/step
0.30548412 0.78895813 0.8610301

 Epoch 5/50
140/140 [==============================] - 1190s 8s/step
0.7125195 0.8017621 0.86749285

 Epoch 6/50
140/140 [==============================] - 1193s 9s/step
0.4663728 0.82300884 0.879017

 Epoch 7/50
140/140 [==============================] - 1190s 8s/step
0.41514367 0.8300181 0.87511915

 Epoch 8/50
140/140 [==============================] - 1197s 9s/step
0.21567994 0.8456679 0.8700093

 Epoch 9/50
140/140 [============

In [97]:
# Get a batch of test data
test_input, test_val, y_true = test_data.as_numpy_iterator().next()

In [98]:
y_hat = siamese_model.predict([test_input, test_val])

y_hat

1/1 [==============================] - 2s 2s/step


array([[0.50027674],
       [0.49971431],
       [0.49994934],
       [0.49949452],
       [0.500532  ],
       [0.5010445 ],
       [0.50052357],
       [0.49936655],
       [0.49959302],
       [0.50010735],
       [0.5002731 ],
       [0.4992336 ],
       [0.49985513],
       [0.50063056],
       [0.5004076 ],
       [0.49934584]], dtype=float32)

In [108]:
# Post processing the results 
[1 if prediction > 0.5 else 0 for prediction in y_hat ]

[1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]

In [109]:
y_true

array([0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0.],
      dtype=float32)

In [110]:
  # Creating a metric object 
m = Recall()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

0.54545456

In [111]:
# Creating a metric object 
m = Precision()

# Calculating the recall value 
m.update_state(y_true, y_hat)

# Return Recall Result
m.result().numpy()

0.75

In [103]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
    yhat = siamese_model.predict([test_input, test_val])
    r.update_state(y_true, yhat)
    p.update_state(y_true,yhat) 

print(r.result().numpy(), p.result().numpy())

1/1 [==============================] - 2s 2s/step
0.72040814 0.5


In [116]:
# Save weights
# siamese_model.save('siamesemodel_common_negative.h5')

In [117]:
L1Dist

__main__.L1Dist

In [120]:
# Reload model 
# siamese_model = tf.keras.models.load_model('siamesemodel_Unique_negative.h5',
siamese_model = tf.keras.models.load_model('siamesemodel_common_negative.h5',
    custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [121]:
# Make predictions with reloaded model
siamese_model.predict([test_input, test_val])

1/1 [==============================] - 2s 2s/step


array([[0.50015163],
       [0.5004492 ],
       [0.50052834],
       [0.50033695],
       [0.5001633 ],
       [0.5000724 ],
       [0.5001251 ],
       [0.500564  ],
       [0.5008859 ],
       [0.500191  ],
       [0.50051737],
       [0.5004922 ],
       [0.49993512],
       [0.49980846],
       [0.4999372 ],
       [0.5002964 ]], dtype=float32)

In [ ]:
# # View model summary
# siamese_model.summary()

In [39]:

name=[]
# Function to mark attendance in a CSV file
def mark_attendance(name):
    timestamp = time.strftime("%Y-%m-%D %H:%M:%S")
    with open('attendance.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, timestamp])

# Main function
def main():
    # Set paths and parameters
    dataset_path = 'path_to_dataset'
    input_shape = (100, 100, 3)  # Adjust to your image dimensions
    siamese_model = create_siamese_model(input_shape)
    siamese_model.load_weights('siamese_model_weights.h5')  # Load trained model weights
    
    # Iterate through dataset folders and classify attendance
    for folder_name in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder_name)
        if os.path.isdir(folder_path):
            images = os.listdir(folder_path)
            if len(images) > 0:
                reference_image_path = os.path.join(folder_path, images[0])
                reference_image = load_and_preprocess_image(reference_image_path, input_shape[:2])
                
                for image_name in images[1:]:
                    image_path = os.path.join(folder_path, image_name)
                    test_image = load_and_preprocess_image(image_path, input_shape[:2])
                    
                    distance = siamese_model.predict([[reference_image], [test_image]])[0]
                    if distance < 0.5:  # Adjust this threshold as needed
                        mark_attendance(folder_name)
                        break  # Mark attendance and move to the next person

if name == "main":
    main()

In [1]:
def classify(model, detection_threshold, verification_threshold):
    #d = {'amisha': 0, 'atish': 0, 'ayush': 0, 'balaji': 0, 'manish': 0, 'misbah': 0, 'raja': 0, 'vatsal': 0}
    d_name = {'amisha':[], 'atish': [], 'ayush': [], 'balaji': [], 'manish': [], 'misbah': [], 'raja': [], 'vatsal': []}
    d_verify = {'amisha': False, 'atish': False, 'ayush': False, 'balaji': False, 'manish': False, 'misbah': False, 'raja': False, 'vatsal': False}
    for image in os.listdir(os.path.join('classify','verification')):
        #input_image.jpg naam hai file ka that is captuerd from the webcam it could be changed
        input_img = preprocess(os.path.join('classify', 'input', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('classify','verification', image))
        # Make Predictions
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        # result = model.predict([input_img, validation_img])
        #extracting name of verification images
        
        L = image.split("_")
        #results.append(result)
        d_name[L[0]].append(result)
# need to run a loop for all people jinka data hai for the below operations


    for i in d_name:
        length = 0
        detection = np.sum(np.array(d_name[i]) > detection_threshold)
        # verification = detection / len(os.listdir(os.path.join('classify', 'verification')))
        for photo in os.listdir(os.path.join('classify', 'verification')):
            L = photo.split("_")
            if (L[0]==i):
                length+=1
        verification = detection / length
        #the length above check karne ka loop hai because multiple people ki images hai
        d_verify[i] = verification < verification_threshold

    #return results, verified
    return d_name, d_verify


In [2]:
d_result, d_verify = classify(siamese_model,0.5,0.99999)
d_verify

NameError: name 'siamese_model' is not defined

In [ ]:
cap = cv2.VideoCapture(4)
while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[120:120+250,200:200+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        # Save input image to application_data/input_image folder 
#         hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         h, s, v = cv2.split(hsv)

#         lim = 255 - 10
#         v[v > lim] = 255
#         v[v <= lim] -= 10
        
#         final_hsv = cv2.merge((h, s, v))
#         img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
        # Run verification
        results, verified = verify(siamese_model, 0.5, 0.5)
        print(verified)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [ ]:

# name=[]
# # Function to mark attendance in a CSV file
# def mark_attendance(name):
#     timestamp = time.strftime("%Y-%m-%D %H:%M:%S")
#     with open('attendance.csv', mode='a', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow([name, timestamp])

# # Main function
# def main():
#     # Set paths and parameters
#     dataset_path = 'path_to_dataset'
#     input_shape = (100, 100, 3)  # Adjust to your image dimensions
#     siamese_model = create_siamese_model(input_shape)
#     siamese_model.load_weights('siamese_model_weights.h5')  # Load trained model weights
    
#     # Iterate through dataset folders and classify attendance
#     for folder_name in os.listdir(dataset_path):
#         folder_path = os.path.join(dataset_path, folder_name)
#         if os.path.isdir(folder_path):
#             images = os.listdir(folder_path)
#             if len(images) > 0:
#                 reference_image_path = os.path.join(folder_path, images[0])
#                 reference_image = load_and_preprocess_image(reference_image_path, input_shape[:2])
                
#                 for image_name in images[1:]:
#                     image_path = os.path.join(folder_path, image_name)
#                     test_image = load_and_preprocess_image(image_path, input_shape[:2])
                    
#                     distance = siamese_model.predict([[reference_image], [test_image]])[0]
#                     if distance < 0.5:  # Adjust this threshold as needed
#                         mark_attendance(folder_name)
#                         break  # Mark attendance and move to the next person

# if name == "main":
#     main()

In [ ]:
# #  OpenCV Real Time Verification

# cap = cv2.VideoCapture(0)
# while cap.isOpened():
#     ret, frame = cap.read()
#     frame = frame[235:235+250,515:250+515]
    
#     cv2.imshow('Verification', frame)
    
#     # Verification trigger
#     if cv2.waitKey(1) & 0xFF == ord('v'):
#         # Save input image to classify/input_image folder 
#         # hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         # h, s, v = cv2.split(hsv)

#         # lim = 255 - 10
#         # v[v > lim] = 255
#         # v[v <= lim] -= 10
        
#         # final_hsv = cv2.merge((h, s, v))
#         # img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)

#         cv2.imwrite(os.path.join('classify', 'input', 'input_image.jpg'), frame)
#         # Run verification
#         d_result, d_verify = classify(siamese_model,0.5,0.999999)
#         print(d_verify)
    
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break
# cap.release()
# cv2.destroyAllWindows()

In [ ]:
# # Function to mark attendance in a txt file
# def mark_attendance(name):
#     timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
#     with open('attendance.txt', mode='a', newline='') as file:
#         writer = file.writer(file)
#         writer.writerow([name, timestamp])

# # Main function
# def main():
#     # Set paths and parameters
#     dataset_path = 'path_to_dataset'
#     input_shape = (128, 128, 3)  # Adjust to your image dimensions
#     siamese_model = create_siamese_model(input_shape)
#     siamese_model.load_weights('siamese_model_weights.h5')  # Load trained model weights
    
#     # Iterate through dataset folders and classify attendance
#     for folder_name in os.listdir(dataset_path):
#         folder_path = os.path.join(dataset_path, folder_name)
#         if os.path.isdir(folder_path):
#             images = os.listdir(folder_path)
#             if len(images) > 0:
#                 reference_image_path = os.path.join(folder_path, images[0])
#                 reference_image = load_and_preprocess_image(reference_image_path, input_shape[:2])
                
#                 for image_name in images[1:]:
#                     image_path = os.path.join(folder_path, image_name)
#                     test_image = load_and_preprocess_image(image_path, input_shape[:2])
                    
#                     distance = siamese_model.predict([[reference_image], [test_image]])[0]
#                     if distance < 0.5:  # Adjust this threshold as needed
#                         mark_attendance(folder_name)
#                         break  # Mark attendance and move to the next person

# if name == "main":
#     main()